In [1]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

!conda install -c conda-forge geopy --yes 
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

# for webscraping import Beautiful Soup 
from bs4 import BeautifulSoup

import xml

!conda install -c conda-forge folium=0.5.0 --yes 
import folium # map rendering library

print('Libraries imported.')

Solving environment: done

# All requested packages already installed.

Solving environment: done

# All requested packages already installed.

Libraries imported.


In [ ]:
# Read the data for examination (Source: http://landregistry.data.gov.uk/)
df_ppd = pd.read_csv("Price_Paid_Data.csv")

# Assign meaningful column names
df_ppd.columns = ['TUID', 'Price', 'Date_Transfer', 'Postcode', 'Prop_Type', 'Old_New', 'Duration', 'PAON', \
                  'SAON', 'Street', 'Locality', 'Town_City', 'District', 'County', 'PPD_Cat_Type', 'Record_Status']

In [ ]:
df_ppd['Date_Transfer'] = df_ppd['Date_Transfer'].apply(pd.to_datetime)

df_ppd.drop(df_ppd[df_ppd.Date_Transfer.dt.year < 2016].index, inplace=True)

df_ppd.sort_values(by=['Date_Transfer'],ascending=[False],inplace=True)

In [ ]:
df_ppd_london = df_ppd.query("Town_City == 'LONDON'")
streets = df_ppd_london['Street'].unique().tolist()

In [ ]:
df_grp_price = df_ppd_london.groupby(['Street'])['Price'].mean().reset_index()
df_grp_price.columns = ['Street', 'Avg_Price']

In [ ]:
df_affordable = df_grp_price.query("(Avg_Price >= 2200000) & (Avg_Price <= 2500000)")
df_affordable

In [ ]:
london_data = pd.read_csv("Street Coordinates London.csv")
london_data['Street'] = london_data['Street'].str.replace(', LONDON', '')

In [ ]:
df_pref_loc = pd.merge(london_data, df_affordable, on=['Street'], how='inner')
df_pref_loc.head(10)

In [ ]:
address = 'London, UK'
geolocator = Nominatim()
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of London City are {}, {}.'.format(latitude, longitude))

In [ ]:
map_london = folium.Map(location=[latitude, longitude], zoom_start=11)

for lat, lng, price, street in zip(df_pref_loc['Latitude'], df_pref_loc['Longitude'], df_pref_loc['Avg_Price'], df_pref_loc['Street']):
    label = '{}, {}'.format(street, price)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_london)  
    
map_london

In [ ]:
CLIENT_ID = '******' # your Foursquare ID
CLIENT_SECRET = '*******' # your Foursquare Secret
VERSION = '2020***' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

In [ ]:
def getNearbyVenues(names, latitudes, longitudes, radius=500, LIMIT=100):

    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
        
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&31={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Street', 
                  'Street Latitude', 
                  'Street Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [ ]:
location_venues = getNearbyVenues(names=df_pref_loc['Street'],
                                   latitudes=df_pref_loc['Latitude'],
                                   longitudes=df_pref_loc['Longitude']

In [ ]:
location_venues

In [ ]:
location_venues.groupby('Street').count()

In [ ]:
def getNearbyFacility(names, latitudes, longitudes, radius=5000):
    facility_list=[]
    radius = 5000
    LIMIT = 100
    categories = '4bf58dd8d48988d196941735,58daa1558bbb0b01f18ec1f7,4bf58dd8d48988d13d941735,4f4533804b9074f6e4fb0105,4bf58dd8d48988d118951735'
    
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        url = 'https://api.foursquare.com/v2/venues/search?&categoryId={}&client_id={}&client_secret={}&v={}&31={},{}&radius={}&limit={}'.format(
        categories,
        CLIENT_ID, 
        CLIENT_SECRET, 
        VERSION, 
        lat, 
        lng, 
        radius, 
        LIMIT)

        results = requests.get(url).json()['response']

        facility_list.append([(
            name,
            facility['name'],
            facility['categories'][0]['name'],
            facility['location']['distance'], 
            facility['location']['lat'],
            facility['location']['lng']) for facility in results['venues']])

    nearby_facility = pd.DataFrame([item for f_list in facility_list for item in f_list])
    nearby_facility.columns = ['Street Name',
                  'Facility Name',             
                  'Facility Category', 
                  'Distance', 
                  'Facility Latitude', 
                  'Facility Longitude']
    
    return(nearby_facility)

In [ ]:
location_facility = getNearbyFacility(names=df_pref_loc['Street'],
                                      latitudes=df_pref_loc['Latitude'],
                                      longitudes=df_pref_loc['Longitude']
                                  )

In [ ]:
location_facility

In [ ]:
location_facility.groupby('Street Name').count()